### Построение эфемерид гравитационного микролинзирования

Программа построена на функциях и классах, в качестве входных данных принимает ID линзы и источника из Gaia dr3, массу линзы с ошибкой и момент времени, на который надо рассчитать эфемериды. 
Исходные данные вводятся в самом низу программы.

На выходе получаем угловое расстояние между звездами, радиус Эйнштейна и эфемериды микролинзирования для разрешенного, неразрешенного и частично разрешенного случая.

In [26]:
import pyvo as vo
import ssl
import numpy as np
from math import *
from astroquery.gaia import Gaia
from datetime import datetime, date
from astropy.time import Time
from astroquery.jplhorizons import Horizons
from zero_point import zpt
zpt.load_tables()

In [27]:
def get_earth_location(t):
    # строго говоря, возвращает -E, если считать, что E - это вектор барицентрического положения Земли
    JD = Time(t, format='jyear').jd
    obj = Horizons(id='ssb', id_type='majorbody', location='3', epochs=JD)
    ev= obj.vectors(refplane='earth')
    E = np.array([ ev['x'][0], ev['y'][0], ev['z'][0]])
    return E

In [28]:
def equatorial_coordinates(ksi, eta, RA, Dec):
    # all parametrs in radians
    x, y, z = np.dot(np.array([[-sin(RA), -cos(RA) * sin(Dec), cos(RA) * cos(Dec)],
                              [cos(RA), -sin(RA) * sin(Dec), sin(RA) * cos(Dec)],
                              [0, cos(Dec), sin(Dec)]]),
                     np.array([ksi, eta, 1])) / sqrt(1 + ksi * ksi + eta * eta)
    ra = atan2(y, x)
    dec = atan2(z, sqrt(x * x + y * y))
    if ra < 0:
        ra += 2 * pi
    return ra, dec

In [29]:
def rad2mas(arr):
    return np.degrees(arr) * 3600000

def nan2zero(value):
    if isnan(value):
        return 0
    return value

In [30]:
class AstrometricCovariationMatrix(object):
    # sigmas ra, dec, plx - mas; sigmas pm_ra, pm_dec - mas/year
    def __init__(self, sigma_ra, sigma_dec, sigma_plx, sigma_pm_ra, sigma_pm_dec, \
                 ra_dec_corr, ra_parallax_corr, ra_pmra_corr, ra_pmdec_corr, \
                 dec_parallax_corr, dec_pmra_corr, dec_pmdec_corr, \
                 parallax_pmra_corr, parallax_pmdec_corr, \
                 pmra_pmdec_corr):
        self.matrix = np.zeros((5, 5))
        self.matrix[0, 0] = sigma_ra ** 2
        self.matrix[1, 1] = sigma_dec ** 2
        self.matrix[2, 2] = sigma_plx ** 2
        self.matrix[3, 3] = sigma_pm_ra ** 2
        self.matrix[4, 4] = sigma_pm_dec ** 2
        self.matrix[0, 1] = ra_dec_corr * sigma_ra * sigma_dec
        self.matrix[0, 2] = ra_parallax_corr * sigma_ra * sigma_plx
        self.matrix[0, 3] = ra_pmra_corr * sigma_ra * sigma_pm_ra
        self.matrix[0, 4] = ra_pmdec_corr * sigma_ra * sigma_pm_dec
        self.matrix[1, 2] = dec_parallax_corr * sigma_dec * sigma_plx
        self.matrix[1, 3] = dec_pmra_corr * sigma_dec * sigma_pm_ra
        self.matrix[1, 4] = dec_pmdec_corr * sigma_dec * sigma_pm_dec
        self.matrix[2, 3] = parallax_pmra_corr * sigma_plx * sigma_pm_ra
        self.matrix[2, 4] = parallax_pmdec_corr * sigma_plx * sigma_pm_dec
        self.matrix[3, 4] = pmra_pmdec_corr * sigma_pm_ra * sigma_pm_dec
        for i in range(1, 5):
            for j in range(i):
                self.matrix[i, j] = self.matrix[j, i]
    
    def __getitem__(self, item):
        return self.matrix.__getitem__(item)
    
    def __setitem__(self, key, value):
        return self.matrix.__setitem__(key, value)
    
    def __iter__(self):
        return self.matrix.__iter__()
    
    def __array__(self, dtype=None, copy=None):
        return self.matrix.__array__(self, dtype=dtype, copy=copy)

In [31]:
class AstrometricParameters(object):
    # ra, dec - degrees; plx - mas; pm_ra, pm_dec - mas/year
    # rad_vel, sigma rad_vel - km/s
    def __init__(self, ra, dec, plx, pm_ra, pm_dec, pmrad_vel, epoch, ready=False):
        # во внутреннем представлении: ra, dec, plx, sigmas ra, dec, plx - rad; pm_ra, pm_dec - rad/year
        #                              rad_vel - AU/year
        A_v = 4.740470446
        if ready:
            self.ra = ra
            self.dec = dec
            self.plx = plx
            self.pm_ra = pm_ra
            self.pm_dec = pm_dec
            self.pmrad_vel = pmrad_vel
            if self.plx == 0:
                self.rad_vel = 0
            else:
                self.rad_vel = self.pmrad_vel/self.plx
        else:
            self.ra = radians(ra)
            self.dec = radians(dec)
            self.plx = radians(plx / 3600000)
            self.pm_ra = radians(pm_ra / 3600000)
            self.pm_dec = radians(pm_dec / 3600000)
            self.pmrad_vel = radians(pmrad_vel/A_v/3.6e6)
            if self.plx == 0:
                self.rad_vel = 0
            else:
                self.rad_vel = self.pmrad_vel/self.plx  # AU/year
        self.epoch = epoch
    
    def __repr__(self):
        return f'''        ra = {degrees(self.ra)}°; dec = {degrees(self.dec)}°;
        parallax = {degrees(self.plx) * 3600000} mas;
        pm_ra = {degrees(self.pm_ra) * 3600000} mas/year; pm_dec = {degrees(self.pm_dec) * 3600000} mas/year;
        rad_vel = {self.rad_vel * 149597870.7 / 31556925.445} km/s\n'''
    
    def __text__(self):
        return self.__repr__()
    
    

    def to_un_vector(self):
        r = np.array([cos(self.dec) * cos(self.ra), cos(self.dec) * sin(self.ra), sin(self.dec)]) 
        return r
   
    
    def epoch_transformation(self, t, eq=False, calculate_E=True, E=np.array([0, 0, 0])):
        if eq:
            xi = self.pm_ra*(t - self.epoch)
            eta = self.pm_dec*(t - self.epoch)
            ra, dec = equatorial_coordinates(xi, eta, self.ra, self.dec)
            return AstrometricParameters(ra, dec, self.plx, self.pm_ra, self.pm_dec, 0.0, t, ready=True)
        else:
            p = np.array([-sin(self.ra), cos(self.ra), 0])
            q = np.array([-sin(self.dec)*cos(self.ra), -sin(self.dec)*sin(self.ra), cos(self.dec)])
            r = np.array([cos(self.dec) * cos(self.ra), cos(self.dec) * sin(self.ra), sin(self.dec)])

            MU = p*self.pm_ra + q*self.pm_dec 
            mu = np.linalg.norm(MU) 
            dt = t - self.epoch
            
            f_d = (1 + 2*self.pmrad_vel*dt + (mu**2 + self.pmrad_vel**2)*dt**2)**(-1/2)
            r_b = (r*(1 + self.pmrad_vel*dt) + MU*dt)
            
            plx1 = self.plx*f_d 
            
            if calculate_E:
                E = get_earth_location(t) 
            
            r_b = r_b + E*plx1
            
            ra = atan2(r_b[1], r_b[0])
            dec = atan2(r_b[2], sqrt(r_b[0] ** 2 + r_b[1] ** 2))
            if ra < 0:
                ra += 2 * pi

            MU1 = f_d**3 * (MU*(1 + self.pmrad_vel*dt) - r*mu**2*dt)
            pm_dec = MU1[2] / cos(dec)
            pm_ra = (MU1[1] + sin(dec) * sin(ra) * pm_dec) / cos(ra)
            mu_r = f_d**2 * (self.pmrad_vel + (self.pmrad_vel**2 + mu**2)*dt)
            return AstrometricParameters(ra, dec, plx1, pm_ra, pm_dec, mu_r, t, ready=True)

    
    
   

    def geocentric_angdist(self, other, eq = False, calculate_E=True, E=np.array([0, 0, 0])):
        # self and other must have the same epoch!
        if not abs(self.epoch - other.epoch) < 1e-6:
            raise ValueError("Astrometric Parameters must have the same epoch!")
        
        if eq:
            if calculate_E:
                E = get_earth_location(self.epoch)
            r0 = self.to_un_vector()
            r1 = other.to_un_vector()
            
            r1 = r1 + E*other.plx
            r1 = r1 / np.linalg.norm(r1)
        
            r0 = r0 + E*self.plx
            r0 = r0 / np.linalg.norm(r0)
            
        else:
            r0 = self.to_un_vector()
            r1 = other.to_un_vector()
            
        return degrees(2 * asin(0.5 * np.linalg.norm(r1 - r0))) * 3600000

In [32]:
class AstrometricParametersSample(object):
    def __init__(self, ras, decs, parallaxes, prop_motions_ra, prop_motions_dec, pmrad_vels, epochs):
        if len(ras) != len(decs) or len(ras) != len(parallaxes) or \
        len(ras) != len(prop_motions_ra) or len(ras) != len(prop_motions_dec) or \
        len(ras) != len(pmrad_vels) or len(ras) != len(epochs):
            raise ValueError("Lengths of all arrays must be equal")
        self.ras = ras
        self.decs = decs
        self.parallaxes = parallaxes
        self.prop_motions_ra = prop_motions_ra
        self.prop_motions_dec = prop_motions_dec
        self.pmrad_vels = pmrad_vels
        self.epochs = epochs
        self.size = len(ras)
    
    def __getitem__(self, item):
        return AstrometricParameters(self.ras[item], self.decs[item], self.parallaxes[item], \
                                     self.prop_motions_ra[item], self.prop_motions_dec[item], self.pmrad_vels[item], \
                                     self.epochs[item], ready=True)
    
    def __len__(self):
        return self.size
    
    def __repr__(self):
        return f"Sample of {len(self)} sets of astrometric parameters"
    
    def __text__(self):
        return self.__repr__()
    
    @classmethod
    def from_list(cls, lst):
        size = len(lst)
        ras = np.zeros(size)
        decs = np.zeros(size)
        parallaxes = np.zeros(size)
        prop_motions_ra = np.zeros(size)
        prop_motions_dec = np.zeros(size)
        pmrad_vels = np.zeros(size)
        epochs = np.zeros(size)
        for i in range(size):
            ras[i] = lst[i].ra
            decs[i] = lst[i].dec
            parallaxes[i] = lst[i].plx
            prop_motions_ra[i] = lst[i].pm_ra
            prop_motions_dec[i] = lst[i].pm_dec
            pmrad_vels[i] = lst[i].pmrad_vel
            epochs[i] = lst[i].epoch
        return cls(ras, decs, parallaxes, prop_motions_ra, prop_motions_dec, pmrad_vels, epochs)
    
    @classmethod
    def from_astrometric_parameters_with_covariation_matrix(cls, sample_size, ap):
        mean_value = [ap.ra, ap.dec, ap.plx, ap.pm_ra, ap.pm_dec, ap.pmrad_vel]
        ras = np.zeros(sample_size)
        decs = np.zeros(sample_size)
        parallaxes = np.zeros(sample_size)
        prop_motions_ra = np.zeros(sample_size)
        prop_motions_dec = np.zeros(sample_size)
        pmrad_vels = np.zeros(sample_size)
        epochs = np.zeros(sample_size)
        for i in range(sample_size):
            ras[i], decs[i], parallaxes[i], prop_motions_ra[i], prop_motions_dec[i], pmrad_vels[i] = \
            np.random.multivariate_normal(mean_value, ap.covariation_matrix)
            epochs[i] = ap.epoch
        return cls(ras, decs, parallaxes, prop_motions_ra, prop_motions_dec, pmrad_vels, epochs)
    
    def transform_all(self, epoch, eq = False, calculate_E = True, E = np.array([0,0,0])):
        new_parameters = []
        for ap in self:
            new_parameters.append(ap.epoch_transformation(epoch,  eq = eq, calculate_E = calculate_E, E = E))
        return AstrometricParametersSample.from_list(new_parameters)

In [33]:
class AstrometricParametersWithCovariation(AstrometricParameters):
    # ra, dec - degrees; plx - mas; pm_ra, pm_dec - mas/year
    # rad_vel, sigma rad_vel - km/s
    # elements of the covariation matrix - rad^2
    def __init__(self, ra, dec, plx, pm_ra, pm_dec, ap_cov_matrix, pmrad_vel, sigma_rad_vel, epoch, ready=False):
        super().__init__(ra, dec, plx, pm_ra, pm_dec, pmrad_vel, epoch, ready=ready)
        if ready:
            srv = sigma_rad_vel
        else:
            srv = sigma_rad_vel / 4.740470446
        self.covariation_matrix = np.zeros((6, 6))
        for i in range(5):
            for j in range(5):
                self.covariation_matrix[i, j] = ap_cov_matrix[i, j]
        for i in range(5):
            self.covariation_matrix[i, 5] = ap_cov_matrix[i, 2] * self.rad_vel
            self.covariation_matrix[5, i] = self.covariation_matrix[i, 5]
        self.covariation_matrix[5, 5] = ap_cov_matrix[2, 2] * (self.rad_vel ** 2 + srv ** 2) + (self.plx * srv) ** 2
    
    def make_sample(self, sample_size):
        return AstrometricParametersSample.from_astrometric_parameters_with_covariation_matrix(sample_size, self)
    
    def __repr__(self):
        return super().__repr__() + "        covariation matrix is given"

    def estimate_parallax_with_gaia_stars(self, g_mag, delta_mag=0.1, init_radius=0.5, max_radius=5.0, min_n_stars=101):
        radius = init_radius
        ra = self.ra
        dec = self.dec
        it = 0
        ok = False
        while not ok:
            it = it + 1
            ssl._create_default_https_context = ssl._create_unverified_context
            job = Gaia.launch_job_async(f"SELECT ra, dec, parallax FROM gaiadr3.gaia_source WHERE CONTAINS(POINT('ICRS',\
                                          gaiadr3.gaia_source.ra,gaiadr3.gaia_source.dec),\
                                          CIRCLE('ICRS',{ra},{dec},{radius}))=1 \
                                          AND phot_g_mean_mag>{g_mag - delta_mag} AND  phot_g_mean_mag<{g_mag + delta_mag}")
            res = job.get_results()
            res_plx = res["parallax"]
            res_plx = res_plx[~np.isnan(res_plx)]
            n_stars = len(res_plx)
            # print(f"Iteration {it}: N_stars = {n_stars}; radius = {radius} deg")
            new_radius = radius * sqrt(1.25 * min_n_stars / (n_stars + 1))
            if n_stars < min_n_stars and radius < max_radius - 0.01:
                if new_radius > max_radius:
                    radius = max_radius
                else:
                    radius = new_radius
            else:
                if n_stars < 0.2 * min_n_stars:
                    print("Cannot estimate parallax statistically. Assume plx = 0, sigma_plx = 3.0 mas")
                    return 0.0, 3.0, -1, radius
                else:
                    print(f"Parallax estimated using {n_stars} stars in radius of {radius} degrees with G = {g_mag} +- {delta_mag}")
                    return np.mean(res_plx), np.std(res_plx), n_stars, radius

    
    @classmethod
    def from_gaia(cls, result, source_id, eq=False):
        
        ra = float(result["ra"])
        dec = float(result["dec"])
        
        pm_ra = float(result["pmra"])
        if isnan(pm_ra):
            try:
                job = Gaia.launch_job_async(f"SELECT * FROM gaiadr2.gaia_source WHERE source_id = {source_id}")
                res = job.get_results()
                pm_ra = 3.6e6*(ra - float(res['ra']))*cos(radians(dec))/(2016.0 - 2015.5)
            except Exception:
                pm_ra = 0
             
        pm_dec = float(result["pmdec"])
        if isnan(pm_dec):
            try:
                pm_dec = 3.6e6*(dec - float(res['dec']))/(2016.0 - 2015.5)
            except Exception:
                pm_dec = 0

        
        plx = float(result["parallax"])
        plx_err = radians(float(result["parallax_error"]) / 3600000)
        
            
        if eq:
            if isnan(plx):
                plx = 4.74*sqrt(pm_ra**2 + pm_dec**2)/75
                plx_err = radians(3.0/3.6e6)
            if plx < 0:
                plx = 0

        if not eq:
            if isnan(plx):
                plx, plx_err, __, ___ = self.estimate_parallax_with_gaia_stars(float(result["phot_g_mean_mag"]))
                plx_err = radians(plx_err / 3600000)
            else:
                cor = zpt.get_zpt(result['phot_g_mean_mag'], result['nu_eff_used_in_astrometry'], result['pseudocolour'], result['ecl_lat'], result['astrometric_params_solved'], _warnings=True)
                plx = plx - cor
            
                
        rad_vel = nan2zero(float(result["radial_velocity"]))
        sigma_rad_vel = float(result["radial_velocity_error"])
        if isnan(sigma_rad_vel):
            sigma_rad_vel = 75
            
        if rad_vel == 0:
            if float(result['phot_g_mean_mag']) < 17.5:
                tap = vo.dal.TAPService("https://gaia.aip.de/tap")
                vr_estim = tap.run_sync(f"SELECT source_id, sample_mean, sample_std FROM gaiadr3_contrib.estimates_gdr3_rv WHERE source_id  = {source_id}")
                if float(vr_estim['sample_std']) <= 75:
                    rad_vel = float(vr_estim['sample_mean'])
                    sigma_rad_vel = float(vr_estim['sample_std'])
        pmrad_vel = rad_vel*plx
        
        
        ra_err = radians(float(result["ra_error"]) / 3600000)
        dec_err = radians(float(result["dec_error"]) / 3600000)
        
            
        pm_ra_err = radians(float(result["pmra_error"]) / 3600000)
        if isnan(pm_ra_err):
            if eq:
                pm_ra_err = radians(3.0/3.6e6)
            else:
                try:
                    pm_ra_err = sqrt(ra_err**2*cos(radians(dec))**2 + radians(res['ra_error']/3.6e6)**2 * cos(radians(dec))**2 + radians(ra - float(res['ra']))**2*sin(radians(dec)**2))/(2016.0 - 2015.5)
                except Exception:
                    pm_ra_err = radians(5.0/3.6e6)
        pm_dec_err = radians(float(result["pmdec_error"]) / 3600000)
        if isnan(pm_dec_err):
            if eq:
                pm_dec_err = radians(4.0/3.6e6)
            else:
                try:
                    pm_dec_err = sqrt(dec_err**2 + radians(res['dec_error']/3.6e6)**2)/(2016.0 - 2015.5)
                except Exception:
                    pm_dec_err = radians(5.0/3.6e6)
       
            
        ra_dec_corr = float(result["ra_dec_corr"])
        ra_parallax_corr = nan2zero(float(result["ra_parallax_corr"]))
        ra_pmra_corr = nan2zero(float(result["ra_pmra_corr"]))
        ra_pmdec_corr = nan2zero(float(result["ra_pmdec_corr"]))
        dec_parallax_corr = nan2zero(float(result["dec_parallax_corr"]))
        dec_pmra_corr = nan2zero(float(result["dec_pmra_corr"]))
        dec_pmdec_corr = nan2zero(float(result["dec_pmdec_corr"]))
        parallax_pmra_corr = nan2zero(float(result["parallax_pmra_corr"]))
        parallax_pmdec_corr = nan2zero(float(result["parallax_pmdec_corr"]))
        pmra_pmdec_corr = nan2zero(float(result["pmra_pmdec_corr"]))
        ap_cov_matrix = AstrometricCovariationMatrix(ra_err, dec_err, plx_err, pm_ra_err, pm_dec_err, \
                                                     ra_dec_corr, ra_parallax_corr, ra_pmra_corr, ra_pmdec_corr, \
                                                     dec_parallax_corr, dec_pmra_corr, dec_pmdec_corr, \
                                                     parallax_pmra_corr, parallax_pmdec_corr, \
                                                     pmra_pmdec_corr)
        return cls(ra, dec, plx, pm_ra, pm_dec, ap_cov_matrix, pmrad_vel, sigma_rad_vel, 2016.0)




    
    def epoch_transformation(self, t):
        print("Warning: Covariation matrix will be lost when using epoch_transformation method!")
        return super().epoch_transformation(t)

In [34]:
def result_with_conf_interval(array, probability):
    median = np.median(array)
    lbound = np.quantile(array, 0.5 - 0.5 * probability)
    ubound = np.quantile(array, 0.5 + 0.5 * probability)
    return median, ubound - median, lbound - median

In [35]:
class Microlensing(object):
    def __init__(self,t, ap_s, ap_l, M, M_err, source_id, lens_id, eq=False):
        self.conf_interval_probability = 0.67
        self.sample_size = 10000
        self.source_sample0 = ap_s.make_sample(self.sample_size)
        self.lens_sample0 = ap_l.make_sample(self.sample_size)
        
        
        E = get_earth_location(t)
        
        
        
        # теперь приведём все элементы к нужной эпохе
        self.source_sample = self.source_sample0.transform_all(t, eq = eq, calculate_E = False, E=E)
        self.lens_sample = self.lens_sample0.transform_all(t, eq = eq, calculate_E = False, E=E)
        
        
       
        # если считаем как Клютер, то он при отрицательных и не данных нам параллаксах подставляет разные значения в angdist и theta_e
        if eq:
            
            if ap_s.plx == 0:
                job = Gaia.launch_job_async(f"SELECT * FROM gaiaedr3.gaia_source WHERE source_id = {source_id}")
                result = job.get_results()
                plx = radians(float(result["parallax"])/3.6e6)
                plx_err = radians(float(result["parallax_error"])/3.6e6)
                self.source_sample.parallaxes = np.random.normal(plx, plx_err, self.sample_size)
            if isnan(float(result["parallax"])):
                job = Gaia.launch_job_async(f"SELECT * FROM gaiaedr3.gaia_source WHERE source_id = {source_id}")
                result = job.get_results()
                plx = 0
                plx_err = radians(3.0/3.6e6)
                self.source_sample.parallaxes = np.random.normal(plx, plx_err, self.sample_size)
        
        # отбразываем те реализации M-K, где параллакс источника превышает параллакс линзы
        if np.max(self.source_sample.parallaxes) >= np.min(self.lens_sample.parallaxes):
            good_source_sample = []
            good_lens_sample = []

            for i in range(self.sample_size):
                if self.source_sample[i].plx < self.lens_sample[i].plx:
                    good_source_sample.append(self.source_sample[i])
                    good_lens_sample.append(self.lens_sample[i])
            self.source_sample = AstrometricParametersSample.from_list(good_source_sample)
            self.lens_sample = AstrometricParametersSample.from_list(good_lens_sample)
            self.sample_size = len(self.lens_sample)

        
        M_sample = np.random.normal(M, M_err, self.sample_size)
        M_sample = abs(M_sample)
        
        
        # angdist in mas:

        self.d_array = np.array([self.source_sample[_].geocentric_angdist(self.lens_sample[_], eq = eq, calculate_E = False, E=E) for _ in range(self.sample_size)])
        self.d = result_with_conf_interval(self.d_array, self.conf_interval_probability)

        
        
        # const = sqrt((2*1.32712442076*(10**20)/299792458**2)*2*206264.8*10**3/149597870700)
        const = 2.853743709142837
        # theta_e = const * sqrt(M * (plx_l - plx_s))    # радиус конуса Эйнштейна в милисекундах дуги
        self.theta_e_array = const * np.sqrt(M_sample * (rad2mas(self.lens_sample.parallaxes) - rad2mas(self.source_sample.parallaxes)))
        self.theta_e = result_with_conf_interval(self.theta_e_array, self.conf_interval_probability)
        
        #u = d / theta_e                                # прицельный параметр
        self.u_array = self.d_array / self.theta_e_array
        self.u = result_with_conf_interval(self.u_array, self.conf_interval_probability)
        
        # Увеличения потоков для I1, I2:
        # A1 = 0.5 + (u ** 2 + 2) / (2 * u * sqrt(u ** 2 + 4))
        self.A1_array = 0.5 + (self.u_array ** 2 + 2) / (2 * self.u_array * np.sqrt(self.u_array ** 2 + 4))
        self.A1 = result_with_conf_interval(self.A1_array, self.conf_interval_probability)
        self.A2_array = self.A1_array - 1
        self.A2 = result_with_conf_interval(self.A2_array, self.conf_interval_probability)
        
        # общее увеличение системы для темной линзы:
        self.A_array = self.A1_array + self.A2_array
        self.A = result_with_conf_interval(self.A_array, self.conf_interval_probability)
        
        # theta_1 = theta_e * (sqrt(u ** 2 + 4) + u) / 2 # расстояние L − I1 в mas
        self.theta_1_array = self.theta_e_array * (np.sqrt(self.u_array ** 2 + 4) + self.u_array) / 2   # расстояние L − I1 в mas
        self.theta_1 = result_with_conf_interval(self.theta_1_array, self.conf_interval_probability)
        # theta_2 = theta_1 - theta_e * u                # расстояние L − I2 / S − I1 в mas
        self.theta_2_array = self.theta_1_array - self.theta_e_array * self.u_array                     # расстояние L − I2 / S − I1 в mas
        self.theta_2 = result_with_conf_interval(self.theta_2_array, self.conf_interval_probability)
    
    
    def unresolved_microlensing(self, f_l, f_l_err, f_s, f_s_err):
        '''The source images I1 and I2, and the lens L, are all unresolved.
        f_ls - lens to source flux ratio'''

        f_l_sample = np.random.normal(f_l, f_l_err, self.sample_size)
        f_s_sample = np.random.normal(f_s, f_s_err, self.sample_size)
        f_ls_sample = f_l_sample/f_s_sample
        # общее увеличение потока системы в случае дополнительного источника света
        A_lum_array = (self.A_array + f_ls_sample) / (f_ls_sample + 1)
        A_lum = result_with_conf_interval(A_lum_array, self.conf_interval_probability)
        dmag_array = 2.5 * np.array([log10(A_lum_array[_]) for _ in range(self.sample_size)])
        dmag = result_with_conf_interval(dmag_array, self.conf_interval_probability)
        
        # расстояние от линзы до центроида света линзы и источника при микролинзировании в mas
        # theta_mic = (self.A1 * self.theta_1 - self.A2 * self.theta_2) / (self.A1 + self.A2 + f_ls)
        theta_mic_array = (self.A1_array * self.theta_1_array - self.A2_array * self.theta_2_array) / (self.A1_array + self.A2_array + f_ls_sample)
        theta_mic = result_with_conf_interval(theta_mic_array, self.conf_interval_probability)
        
        # расстояние от линзы до центроида света линзы и источника без микролинзирования в mas
        # theta_ls = self.theta_e * self.u / (1 + f_ls)
        theta_ls_array = self.theta_e_array * self.u_array / (1 + f_ls_sample)
        theta_ls = result_with_conf_interval(theta_ls_array, self.conf_interval_probability)
        # смещение центроида света
        # shift_lum = theta_mic - theta_ls
        shift_lum_array = theta_mic_array - theta_ls_array
        shift_lum = result_with_conf_interval(shift_lum_array, self.conf_interval_probability)
        
        #смещение центроида света изображений для темной линзы относительно источника:
        # shift = self.u * self.theta_e / (self.u ** 2 + 2)
        shift_array = self.u_array * self.theta_e_array / (self.u_array ** 2 + 2)
        shift = result_with_conf_interval(shift_array, self.conf_interval_probability)
        
        return {'A_lum': A_lum, 'dm': dmag, 'theta_mic': theta_mic, 'theta_ls': theta_ls, 'centroid_shift': shift, 'centroid_shift_lum' : shift_lum}
    
    
    def partial_microlensing(self, f_l, f_l_err, f_s, f_s_err):
        '''Случай микролинзирования с частичным разрешением, когда 1 изображение и линза неразрешены.
        Принимает на вход потоки источника и линзы.
        Возвращает увеличения потока системы 2 изображения и с линзой,
        угловое расстояние центроида 2 изображения и линзы, отсчитываемое от линзы'''

        f_l_sample = np.random.normal(f_l, f_l_err, self.sample_size)
        f_s_sample = np.random.normal(f_s, f_s_err, self.sample_size)
        f_ls_sample = f_l_sample/f_s_sample
        # A_2l = (self.A2 + f_ls) / f_ls
        A_2l_array = (self.A2_array + f_ls_sample) / f_ls_sample
        A_2l = result_with_conf_interval(A_2l_array, self.conf_interval_probability)

        # theta_2l = self.A2 * self.theta_2 / (self.A2 + f_ls)
        theta_2l_array = self.A2_array * self.theta_2_array / (self.A2_array + f_ls_sample)
        theta_2l = result_with_conf_interval(theta_2l_array, self.conf_interval_probability)
        
        return {'A_2l': A_2l, 'theta_2l': theta_2l}

Подставляем начальные данные для рассчетов углового расстояния, радиуса Эйнштейна и 
эфемерид микролинзирования

In [36]:
# Задаем номера объектов из Gaia dr3
lens_id = 2048157036035036288
source_id = 2048157036035041536
job = Gaia.launch_job_async(f"SELECT * FROM gaiadr3.gaia_source WHERE source_id in {(lens_id, source_id)}")
result = job.get_results()
gaia_lens = result[result['source_id'] == lens_id]
gaia_source = result[result['source_id'] == source_id]

f_l = float(gaia_lens['phot_g_mean_flux'])
f_s = float(gaia_source['phot_g_mean_flux'])
f_l_err = float(gaia_lens['phot_g_mean_flux_error'])
f_s_err = float(gaia_source['phot_g_mean_flux_error'])


# Выбираем метод построения эфемерид
# eq = True -- рассчеты, приближенные к рассчетам Клютера
# eq = False -- наши независимые рассчеты. Сюда можно подставлять 2-параметрические линзы в том числе
eq = False

lens_parameters = AstrometricParametersWithCovariation.from_gaia(gaia_lens, lens_id, eq=eq)
source_parameters = AstrometricParametersWithCovariation.from_gaia(gaia_source, source_id, eq=eq)

# Отображаем астрометрические параметры на эпоху t_ref (2016.0)
print(lens_parameters)
print("\n")
print(source_parameters)
print("\n")

# Подставляем массу линзы с ошибкой и момент времени
service = vo.dal.TAPService("http://dc.g-vo.org/tap")
res1 = service.search(f"SELECT * FROM plc3.data WHERE ob_id = {source_id} and lens_id = {lens_id} ")
t = float(res1['tca'])
M = float(res1['mass'])
M_err = float(res1['err_mass'])



# создаем объект класса
event = Microlensing(t, source_parameters, lens_parameters, M, M_err, source_id, lens_id, eq=eq) 


INFO: Query finished. [astroquery.utils.tap.core]


C:\Users\alsin\AppData\Local\Temp\ipykernel_2356\1766499752.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.plx = radians(plx / 3600000)
C:\Users\alsin\AppData\Local\Temp\ipykernel_2356\1766499752.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.pmrad_vel = radians(pmrad_vel/A_v/3.6e6)
C:\Users\alsin\AppData\Local\Temp\ipykernel_2356\3385565159.py:102: UserWarning: Warning: converting a masked element to nan.
  rad_vel = nan2zero(float(result["radial_velocity"]))
C:\Users\alsin\AppData\Local\Temp\ipykernel_2356\3385565159.py:103: UserWarning: Warning: converting a masked element to nan.
  sigma_rad_vel = float(result["r

        ra = 294.70271659565253°; dec = 35.21399449245036°;
        parallax = 29.654954081425174 mas;
        pm_ra = -9.85981429763143 mas/year; pm_dec = 807.5949606233564 mas/year;
        rad_vel = -104.64321312532968 km/s
        covariation matrix is given


        ra = 294.7025342072433°; dec = 35.216238352979296°;
        parallax = 0.5954787739281854 mas;
        pm_ra = -2.268003947420348 mas/year; pm_dec = -7.303923797460425 mas/year;
        rad_vel = -3.889454786403871 km/s
        covariation matrix is given




In [37]:
# Выводим результаты
print(f"dist = {event.d} mas")
print(f"theta_e: {event.theta_e} mas \n u: {event.u}\n A1: {event.A1}\n A2: {event.A2}\n A: {event.A} \n theta_i1s: {event.theta_2} mas")
print()
print(f"unresolved_microlensing: {event.unresolved_microlensing(f_l, f_l_err, f_s, f_s_err)}")
print()
print(f"partial_microlensing: {event.partial_microlensing(f_l, f_l_err, f_s, f_s_err)}")

dist = (439.556755119967, 0.23920940799638402, -0.23764745551409305) mas
theta_e: (6.8477831253921355, 0.3263036330211504, -0.33470023835999196) mas 
 u: (64.18526158701036, 3.3120850414637175, -2.913284544694939)
 A1: (1.0000000588623026, 1.2012182803999849e-08, -1.0726076249412131e-08)
 A2: (5.88623024766477e-08, 1.201218293500616e-08, -1.0726076177247637e-08)
 A: (1.000000117724605, 2.4024365830044303e-08, -2.1452152276779657e-08) 
 theta_i1s: (0.10664813118484062, 0.01041024746637105, -0.010170852040058659) mas

unresolved_microlensing: {'A_lum': (1.0000000463298322, 9.449293347785215e-09, -8.442974985811702e-09), 'dm': (5.030197500693946e-08, 1.025943947815083e-08, -9.166843316341082e-09), 'theta_mic': (173.0348699424386, 0.10439174561778941, -0.1042928995706518), 'theta_ls': (172.99309798755834, 0.10436912279828903, -0.10451533177197803), 'centroid_shift': (0.10662225365860947, 0.010404924621107384, -0.010166137972401207), 'centroid_shift_lum': (0.041975194027656926, 0.0040938687